In [1]:
import tkinter as tk
import itertools
import numpy
import time
import threading
import pypot
from pypot.creatures import PoppyTorso

# Check for display environment variable (for Linux)
import os
if os.environ.get('DISPLAY', '') == '':
    print('No display found. Using :0')
    os.environ['DISPLAY'] = ':0'

head_z = 36
head_y = 37
l_shoulder_y = 41
l_shoulder_x = 42
l_elbow_y = 54
r_shoulder_y = 51
r_shoulder_x = 52
r_elbow_y = 44
starting_pos = {head_z: 0,
                    head_y: -60,
                    l_shoulder_y: 0,
                    l_shoulder_x: 90,
                    l_elbow_y: 90,
                    r_shoulder_y: 0,
                    r_shoulder_x: -45,
                    r_elbow_y: -90}

# Define EyeAnimation class as provided earlier

def draw_gradient_eye(center_x, center_y, radius, color_start, color_end, canvas, layers=10):
    r_start, g_start, b_start = color_start
    r_end, g_end, b_end = color_end
    for i in range(layers):
        ratio = i / layers
        r = int(r_start + (r_end - r_start) * ratio)
        g = int(g_start + (g_end - g_start) * ratio)
        b = int(b_start + (b_end - b_start) * ratio)
        color = f'#{r:02x}{g:02x}{b:02x}'

        # Gradually reduce the radius for the gradient effect
        current_radius = radius * (1 - i / (layers * 2))
        canvas.create_oval(center_x - current_radius, center_y - current_radius,
                           center_x + current_radius, center_y + current_radius,
                           fill=color, outline="", tags="eye")

# Function to dynamically position and draw the eyes
def draw_eyes(canvas, canvas_width, canvas_height, pulse_factor, eye_radius):
    canvas.delete("eye")  # Clear previous eyes
    # Calculate eye positions based on canvas size
    eye_left_x, eye_left_y = canvas_width * 0.3, canvas_height / 2.5
    eye_right_x, eye_right_y = canvas_width * 0.7, canvas_height / 2.5

    # Define the color gradient (same for both eyes)
    color_start = (30, 144, 255)  # Starting color (light blue)
    color_end = (0, 0, 0)        # Ending color (black)

    # Calculate the radius for the pulsing effect
    pulse_radius = eye_radius + 5 * pulse_factor

    # Draw the left and right eyes with pulsing radius and gradient effect
    draw_gradient_eye(eye_left_x, eye_left_y, pulse_radius, color_start, color_end, canvas, layers=50)
    draw_gradient_eye(eye_right_x, eye_right_y, pulse_radius, color_start, color_end, canvas, layers=50)

# Animate the gradient glow (pulsing effect)
def animate_glow(canvas, canvas_width, canvas_height, eye_radius):
    pulse_time = 0  # Time variable to control the pulsing effect
    pulse_amplitude = 0.05  # How much the radius will change
    pulse_frequency = 1  # Speed of the pulse effect

    def pulse_eyes():
        nonlocal pulse_time
        pulse_time += pulse_frequency
        pulse_factor = numpy.sin(pulse_time)  # Smooth sine wave for pulsing effect
        draw_eyes(canvas, canvas_width, canvas_height, pulse_factor, eye_radius)  # Redraw eyes with pulsing effect
        canvas.after(100, pulse_eyes)  # Schedule the next animation frame

    pulse_eyes()

def callib():
    for motor, id in starting_pos.items():
        dxl_io.set_goal_position({motor: id})
        time.sleep(0.1)
    time.sleep(2)
    
def wave():
    dxl_io.set_goal_position({r_shoulder_y: 90})
    dxl_io.set_goal_position({r_shoulder_x: 0})
    dxl_io.set_goal_position({l_shoulder_y: -90})
    dxl_io.set_goal_position({l_shoulder_x: 35})
    time.sleep(2)
    for i in range(5):
        dxl_io.set_goal_position({r_elbow_y: 0})
        dxl_io.set_goal_position({l_elbow_y: 0})
        time.sleep(1)
        dxl_io.set_goal_position({r_elbow_y: -90})
        dxl_io.set_goal_position({l_elbow_y: 90})
        time.sleep(1)

def hands_up():
    dxl_io.set_goal_position({41: -90})
    dxl_io.set_goal_position({51: 90})
    time.sleep(1)
    dxl_io.set_goal_position({44: -90})
    dxl_io.set_goal_position({54: 90})
    time.sleep(1)
    dxl_io.set_goal_position({42: 0})
    dxl_io.set_goal_position({52: 45})
    time.sleep(2)

def hello():
    dxl_io.set_goal_position({r_shoulder_y: 60})
    dxl_io.set_goal_position({r_elbow_y: 45})
    time.sleep(1)
    dxl_io.set_goal_position({r_elbow_y: -90})
    time.sleep(0.5)
    dxl_io.set_goal_position({r_shoulder_y: 0})
    time.sleep(2)

def T_Pose():
    dxl_io.set_goal_position({r_shoulder_x: 45})
    dxl_io.set_goal_position({l_shoulder_x: 0})
    time.sleep(1)
    dxl_io.set_goal_position({r_elbow_y: 0})
    dxl_io.set_goal_position({l_elbow_y: 0})
    time.sleep(2)

def walkingHands():
    AMP = 100
    FREQ = 0.5
    t0 = time.time()
    while True:
        t = time.time()
        if (t - t0) > 10:
            break

        pos = AMP * numpy.sin(2 * numpy.pi * FREQ * t)
        dxl_io.set_goal_position(dict(zip(ids1, itertools.repeat(pos))))
        print(pos)
        time.sleep(0.02)

def Ready():
    dxl_io.set_goal_position({r_shoulder_x: 45})
    dxl_io.set_goal_position({r_shoulder_y: 90})
    time.sleep(1)
    dxl_io.set_goal_position({r_shoulder_x: -45})
    dxl_io.set_goal_position({r_shoulder_y: 0})
    time.sleep(2)

def Sequence():
    callib()
    wave()
    callib()
    
# Initialize Dynamixel and Tkinter components
if __name__ == '__main__':
    # Dynamixel setup
    ports = pypot.dynamixel.get_available_ports()
    if not ports:
        raise IOError('No port available for Dynamixel communication.')

    port = ports[0]
    dxl_io = pypot.dynamixel.DxlIO(port)
    found_ids = dxl_io.scan()
    if len(found_ids) < 2:
        raise IOError('Connect at least two motors for this test.')

    ids = found_ids[:-1]
    ids1 = found_ids[2:6:3]
    dxl_io.enable_torque(ids)
    dxl_io.set_moving_speed(dict(zip(ids, itertools.repeat(100))))

    # Tkinter setup
    root = tk.Tk()
    root.title("Eye Animation and Servo Control")
    canvas = tk.Canvas(root, width=800, height=600, bg="black")
    canvas.pack()

    # Set up the eyes and animation
    eye_radius = 100  # Base radius for the eyes
    animate_glow(canvas, 800, 600, eye_radius)

    # Run the motor control sequence in a background thread
    threading.Thread(target=Sequence,daemon=True).start()

    # Start the Tkinter main loop
    root.mainloop()
   

No display found. Using :0


In [2]:
import tkinter as tk
import itertools
import numpy
import time
import threading
import pypot.dynamixel
if __name__ == '__main__':
    # Dynamixel setup
    ports = pypot.dynamixel.get_available_ports()
    if not ports:
        raise IOError('No port available for Dynamixel communication.')

    port = ports[0]
    dxl_io = pypot.dynamixel.DxlIO(port)
    found_ids = dxl_io.scan()
    if len(found_ids) < 2:
        raise IOError('Connect at least two motors for this test.')

    ids = found_ids[:-1]
    ids1 = found_ids[2:6:3]
    print(ids)

SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/poppy/pyenv/lib/python3.7/site-packages/pypot/utils/stoppablethread.py", line 130, in _wrapped_target
    self._target()
  File "/home/poppy/pyenv/lib/python3.7/site-packages/pypot/utils/stoppablethread.py", line 205, in run
    make_update_loop(self, self._update)
  File "/home/poppy/pyenv/lib/python3.7/site-packages/pypot/utils/stoppablethread.py", line 175, in make_update_loop
    update_func()
  File "/home/poppy/pyenv/lib/python3.7/site-packages/pypot/dynamixel/controller.py", line 60, in update
    self.set_register(self.synced_motors))
  File "/home/poppy/pyenv/lib/python3.7/site-packages/pypot/dynamixel/controller.py", line 92, in set_register
    getattr(self.io, 'set_{}'.format(self.regname))(dict(zip(ids, va

In [1]:
herborist

NameError: name 'herborist' is not defined